# Predict the sales of a shop using machine learning

## Problem Statement: 

* In this competition you will work with a challenging time-series dataset consisting of daily sales data, kindly provided by one of the largest Ecom firms - 1C Company. 

* We are asking you to predict total sales for every product and store in the next month. By solving this competition you will be able to apply and enhance your data science skills.

## Data Desctiption:

You are provided with daily historical sales data. The task is to forecast the total amount of products sold in every shop for the test set. Note that the list of shops and products slightly changes every month. Creating a robust model that can handle such situations is part of the challenge.

## File descriptions
* sales_train.csv - the training set. Daily historical data from January 2013 to October 2015.
* test.csv - the test set. You need to forecast the sales for these shops and products for November 2015.
* items.csv - supplemental information about the items/products.
* item_categories.csv  - supplemental information about the items categories.
* shops.csv- supplemental information about the shops.

### Data fields
* ID - an Id that represents a (Shop, Item) tuple within the test set
* shop_id - unique identifier of a shop
* item_id - unique identifier of a product
* item_category_id - unique identifier of item category
* item_cnt_day - number of products sold. You are predicting a monthly amount of this measure
* item_price - current price of an item
* date - date in format dd/mm/yyyy
* date_block_num - a consecutive month number, used for convenience. January 2013 is 0, February 2013 is 1,..., October 2015 is 33
* item_name - name of item
* shop_name - name of shop
* item_category_name - name of item category

In [1]:
# !pip install googletrans

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from pandas.plotting import scatter_matrix
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
%matplotlib inline
# from googletrans import Translator

In [3]:
sales = pd.read_csv('sales_train.csv')
item_categories = pd.read_csv('item_categories.csv')
shops = pd.read_csv('shops.csv')
items = pd.read_csv('items.csv')

In [4]:
sales.head()
items.head()
item_categories.head()
shops.head()

,shop_name,shop_id
0,"!Якутск Орджоникидзе, 56 фран",0
1,"!Якутск ТЦ ""Центральный"" фран",1
2,"Адыгея ТЦ ""Мега""",2
3,"Балашиха ТРК ""Октябрь-Киномир""",3
4,"Волжский ТЦ ""Волга Молл""",4


In [5]:
# Merging the dataframes into one
sales_and_items = (pd.merge(sales, items, left_on='item_id', right_on='item_id', how='left'))
sales_and_items = (pd.merge(sales_and_items, item_categories, left_on='item_category_id', right_on='item_category_id', how='left'))
sales = (pd.merge(sales_and_items, shops, left_on='shop_id', right_on='shop_id', how='left'))

In [6]:
sales.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_name,item_category_id,item_category_name,shop_name
0,02.01.2013,0,59,22154,999.00,1.0,ЯВЛЕНИЕ 2012 (BD),37,Кино - Blu-Ray,"Ярославль ТЦ ""Альтаир"""
1,03.01.2013,0,25,2552,899.00,1.0,DEEP PURPLE The House Of Blue Light LP,58,Музыка - Винил,"Москва ТРК ""Атриум"""
2,05.01.2013,0,25,2552,899.00,-1.0,DEEP PURPLE The House Of Blue Light LP,58,Музыка - Винил,"Москва ТРК ""Атриум"""
3,06.01.2013,0,25,2554,1709.05,1.0,DEEP PURPLE Who Do You Think We Are LP,58,Музыка - Винил,"Москва ТРК ""Атриум"""
4,15.01.2013,0,25,2555,1099.00,1.0,DEEP PURPLE 30 Very Best Of 2CD (Фирм.),56,Музыка - CD фирменного производства,"Москва ТРК ""Атриум"""


In [7]:
sales.shape

(2935849, 10)

In [8]:
sales.dtypes

date                   object
date_block_num          int64
shop_id                 int64
item_id                 int64
item_price            float64
item_cnt_day          float64
item_name              object
item_category_id        int64
item_category_name     object
shop_name              object
dtype: object

In [9]:
sales.describe()

,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_category_id
count,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06,2.935849e+06
mean,1.456991e+01,3.300173e+01,1.019723e+04,8.908532e+02,1.242641e+00,4.000138e+01
std,9.422988e+00,1.622697e+01,6.324297e+03,1.729800e+03,2.618834e+00,1.710076e+01
min,0.000000e+00,0.000000e+00,0.000000e+00,-1.000000e+00,-2.200000e+01,0.000000e+00
25%,7.000000e+00,2.200000e+01,4.476000e+03,2.490000e+02,1.000000e+00,2.800000e+01
50%,1.400000e+01,3.100000e+01,9.343000e+03,3.990000e+02,1.000000e+00,4.000000e+01
75%,2.300000e+01,4.700000e+01,1.568400e+04,9.990000e+02,1.000000e+00,5.500000e+01
max,3.300000e+01,5.900000e+01,2.216900e+04,3.079800e+05,2.169000e+03,8.300000e+01


In [10]:
sales.isnull().values.any()

False

In [11]:
# sales.isnull().any()

In [12]:
sales.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 2935849 entries, 0 to 2935848
Data columns (total 10 columns):
 #   Column              Dtype  
---  ------              -----  
 0   date                object 
 1   date_block_num      int64  
 2   shop_id             int64  
 3   item_id             int64  
 4   item_price          float64
 5   item_cnt_day        float64
 6   item_name           object 
 7   item_category_id    int64  
 8   item_category_name  object 
 9   shop_name           object 
dtypes: float64(2), int64(4), object(4)
memory usage: 246.4+ MB


In [13]:
sales['date']= pd.to_datetime(sales['date'])

In [14]:
# sales['item_name'] = sales['item_name'].astype('|S') # unicode encode error occurs

In [15]:
# sales['item_category_name'] = sales['item_category_name'].astype('|S')

In [16]:
# sales['shop_name'] = sales['shop_name'].astype('|S')

In [17]:
# translator = Translator()
# print(translator.translate('Адыгея ТЦ "Мега"'))

In [18]:
# sales['item_name'] = sales['item_name'].apply(translator.translate, src='ru', dest='en').apply(getattr, args=('text',))

In [19]:
# make a deep copy of the data frame
sales_en = sales.loc[:, ['item_name', 'item_category_name', 'shop_name']]

# translate columns' name using rename function
# sales_en.rename(columns=lambda x: translator.translate(x).text, inplace=True)

# translated column names
sales_en.head()
# Index(['level', 'Sex', 'Age', 'administrative status', 'Nationality country',
    #'Geographical area', 'Professional category', 'Level studies'],
    #dtype='object')

,item_name,item_category_name,shop_name
0,ЯВЛЕНИЕ 2012 (BD),Кино - Blu-Ray,"Ярославль ТЦ ""Альтаир"""
1,DEEP PURPLE The House Of Blue Light LP,Музыка - Винил,"Москва ТРК ""Атриум"""
2,DEEP PURPLE The House Of Blue Light LP,Музыка - Винил,"Москва ТРК ""Атриум"""
3,DEEP PURPLE Who Do You Think We Are LP,Музыка - Винил,"Москва ТРК ""Атриум"""
4,DEEP PURPLE 30 Very Best Of 2CD (Фирм.),Музыка - CD фирменного производства,"Москва ТРК ""Атриум"""


In [20]:
# translations = {}
# for column in sales_en.columns:
#     # unique elements of the column
#     unique_elements = sales_en[column].unique()
#     for element in unique_elements:
#         # add translation to the dictionary
#         translations[element] = translator.translate(element).text
    
# print(translations)

## Data Visualization

In [21]:
sales.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_name,item_category_id,item_category_name,shop_name
0,2013-02-01,0,59,22154,999.00,1.0,ЯВЛЕНИЕ 2012 (BD),37,Кино - Blu-Ray,"Ярославль ТЦ ""Альтаир"""
1,2013-03-01,0,25,2552,899.00,1.0,DEEP PURPLE The House Of Blue Light LP,58,Музыка - Винил,"Москва ТРК ""Атриум"""
2,2013-05-01,0,25,2552,899.00,-1.0,DEEP PURPLE The House Of Blue Light LP,58,Музыка - Винил,"Москва ТРК ""Атриум"""
3,2013-06-01,0,25,2554,1709.05,1.0,DEEP PURPLE Who Do You Think We Are LP,58,Музыка - Винил,"Москва ТРК ""Атриум"""
4,2013-01-15,0,25,2555,1099.00,1.0,DEEP PURPLE 30 Very Best Of 2CD (Фирм.),56,Музыка - CD фирменного производства,"Москва ТРК ""Атриум"""


In [22]:
# fig, axs = plt.subplots(1, 3, figsize=(9, 3), sharey=True)
# # axs[0].bar('item_name', 'date_block_num')
# axs[1].scatter('date_block_num', 'item_cnt_day')
# # axs[2].plot('item_name', 'date_block_num')
# fig.suptitle('Categorical Plotting')

### Questions to answer for data analysis

* Q1: Total items sold in a day
* Q2: Total items sold in a month
* Q3: Maximum number of items sold in which month?
* Q4: Maximum items sold in which year
* Q5: Average number of items sold in a day, month and a year.
* Q6: Which shop sold the maximum number of items
* Q7: Average number of items sold by each shop
* Q8: Maximum number of items sold by which shop category
* Q9: Relationship between item price and total items sold 
* Q10: Maximum and average items sold from which category
* Q11: Which item is sold the most and least

### Q1: Total items sold in a day

In [23]:
sales.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_name,item_category_id,item_category_name,shop_name
0,2013-02-01,0,59,22154,999.00,1.0,ЯВЛЕНИЕ 2012 (BD),37,Кино - Blu-Ray,"Ярославль ТЦ ""Альтаир"""
1,2013-03-01,0,25,2552,899.00,1.0,DEEP PURPLE The House Of Blue Light LP,58,Музыка - Винил,"Москва ТРК ""Атриум"""
2,2013-05-01,0,25,2552,899.00,-1.0,DEEP PURPLE The House Of Blue Light LP,58,Музыка - Винил,"Москва ТРК ""Атриум"""
3,2013-06-01,0,25,2554,1709.05,1.0,DEEP PURPLE Who Do You Think We Are LP,58,Музыка - Винил,"Москва ТРК ""Атриум"""
4,2013-01-15,0,25,2555,1099.00,1.0,DEEP PURPLE 30 Very Best Of 2CD (Фирм.),56,Музыка - CD фирменного производства,"Москва ТРК ""Атриум"""


In [24]:
sales_per_day =pd.DataFrame(sales.groupby(sales.date.dt.date)['item_cnt_day'].sum())
sales_per_day.head()

,item_cnt_day
date,
2013-01-01,1951.0
2013-01-02,3817.0
2013-01-03,3846.0
2013-01-04,2574.0
2013-01-05,4863.0


In [25]:
sales_per_day.describe()

,item_cnt_day
count,1034.000000
mean,3528.245648
std,1584.687467
min,1274.000000
25%,2472.250000
50%,3145.000000
75%,4257.000000
max,13343.000000


* Mean sales of items in a day is 3528.246

In [26]:
# sales_per_day.max()

In [27]:
print(sales_per_day[sales_per_day.item_cnt_day == sales_per_day.item_cnt_day.max()])

            item_cnt_day
date                    
2013-12-28       13343.0


- Maximum sales were done on 28 december 2013

In [28]:
print(sales_per_day[sales_per_day.item_cnt_day == sales_per_day.item_cnt_day.min()])

            item_cnt_day
date                    
2015-12-05        1274.0


* Minimum sales were done on 5 December 2015

#### Q2: Total items sold in a month

In [29]:
sales_per_month =pd.DataFrame(sales.groupby(pd.Grouper(key='date',freq='M')).agg({'item_cnt_day':'sum'}))
sales_per_month.head()

,item_cnt_day
date,
2013-01-31,116950.0
2013-02-28,133607.0
2013-03-31,140586.0
2013-04-30,112185.0
2013-05-31,116808.0


In [30]:
sales_per_month.describe()

,item_cnt_day
count,36.000000
mean,101339.055556
std,34134.579781
min,21484.000000
25%,77706.000000
50%,106000.000000
75%,117997.250000
max,184559.000000


* Mean items sold in amonth were 101339.1

In [31]:
print(sales_per_month[sales_per_month.item_cnt_day == sales_per_month.item_cnt_day.max()])

            item_cnt_day
date                    
2013-12-31      184559.0


* Maximum number of items were sold in December 2013

In [32]:
print(sales_per_month[sales_per_month.item_cnt_day == sales_per_month.item_cnt_day.min()])

            item_cnt_day
date                    
2015-12-31       21484.0


* Minimum items were sold in December 2015

### Q3: Total items sold in a year

In [33]:
sales_per_year =pd.DataFrame(sales.groupby(pd.Grouper(key='date',freq='Y')).agg({'item_cnt_day':'sum'}))
sales_per_year

,item_cnt_day
date,
2013-12-31,1562733.0
2014-12-31,1320889.0
2015-12-31,764584.0


* we can see that maximum number of sales were done in the year 2013 and minimum in 2015

In [34]:
sales_per_year.describe()

,item_cnt_day
count,3.000000e+00
mean,1.216069e+06
std,4.092688e+05
min,7.645840e+05
25%,1.042736e+06
50%,1.320889e+06
75%,1.441811e+06
max,1.562733e+06


* Mean sales of items per year is 1.216069e+06

In [35]:
# dates = plt.dates.date2num(list_of_datetimes)
# plt.plot_date(dates, 'items_cnt_day')

In [36]:
# do some visualisations here with date, month and year

In [37]:
# sales.plot(kind='box', subplots=True, layout=(2,2), sharex=False, sharey=False)
# plt.show()

In [38]:
# sales.hist(figsize = (100,100), xlabelsize = 100, ylabelsize = 100)
# plt.show()

In [39]:
# scatter_matrix(sales)
# plt.show()

### Q: Which shop sold the maximum number of items

In [40]:
sales.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_name,item_category_id,item_category_name,shop_name
0,2013-02-01,0,59,22154,999.00,1.0,ЯВЛЕНИЕ 2012 (BD),37,Кино - Blu-Ray,"Ярославль ТЦ ""Альтаир"""
1,2013-03-01,0,25,2552,899.00,1.0,DEEP PURPLE The House Of Blue Light LP,58,Музыка - Винил,"Москва ТРК ""Атриум"""
2,2013-05-01,0,25,2552,899.00,-1.0,DEEP PURPLE The House Of Blue Light LP,58,Музыка - Винил,"Москва ТРК ""Атриум"""
3,2013-06-01,0,25,2554,1709.05,1.0,DEEP PURPLE Who Do You Think We Are LP,58,Музыка - Винил,"Москва ТРК ""Атриум"""
4,2013-01-15,0,25,2555,1099.00,1.0,DEEP PURPLE 30 Very Best Of 2CD (Фирм.),56,Музыка - CD фирменного производства,"Москва ТРК ""Атриум"""


In [41]:
by_shop = sales.groupby('shop_name').agg({'item_cnt_day':'sum'})
by_shop.head()

,item_cnt_day
shop_name,
"!Якутск Орджоникидзе, 56 фран",11705.0
"!Якутск ТЦ ""Центральный"" фран",6311.0
"Адыгея ТЦ ""Мега""",30620.0
"Балашиха ТРК ""Октябрь-Киномир""",28355.0
"Волжский ТЦ ""Волга Молл""",43942.0


In [42]:
by_shop.describe()

,item_cnt_day
count,60.000000
mean,60803.433333
std,57992.901750
min,330.000000
25%,23333.000000
50%,50176.000000
75%,69562.250000
max,310777.000000


### Inferences:
* Total shops are 60.
* Maximum items sold from a single shop is 310777
* Minimum items sold from a single shop is 330
* Mean number of items sold from each shop are 60803.44

In [43]:
print(by_shop[by_shop.item_cnt_day == by_shop.item_cnt_day.max()])

                         item_cnt_day
shop_name                            
Москва ТЦ "Семеновский"      310777.0


In [44]:
print(by_shop[by_shop.item_cnt_day == by_shop.item_cnt_day.min()])

                                       item_cnt_day
shop_name                                          
Новосибирск ТРЦ "Галерея Новосибирск"         330.0


## Machine learning

In [45]:
sales.corr()

,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_category_id
date_block_num,1.000000,0.019273,0.009356,0.095010,0.009402,0.029128
shop_id,0.019273,1.000000,0.029396,-0.024034,-0.005230,0.019380
item_id,0.009356,0.029396,1.000000,-0.134104,0.016650,0.369431
item_price,0.095010,-0.024034,-0.134104,1.000000,0.011197,-0.253554
item_cnt_day,0.009402,-0.005230,0.016650,0.011197,1.000000,0.014768
item_category_id,0.029128,0.019380,0.369431,-0.253554,0.014768,1.000000


In [46]:
sales.head()

,date,date_block_num,shop_id,item_id,item_price,item_cnt_day,item_name,item_category_id,item_category_name,shop_name
0,2013-02-01,0,59,22154,999.00,1.0,ЯВЛЕНИЕ 2012 (BD),37,Кино - Blu-Ray,"Ярославль ТЦ ""Альтаир"""
1,2013-03-01,0,25,2552,899.00,1.0,DEEP PURPLE The House Of Blue Light LP,58,Музыка - Винил,"Москва ТРК ""Атриум"""
2,2013-05-01,0,25,2552,899.00,-1.0,DEEP PURPLE The House Of Blue Light LP,58,Музыка - Винил,"Москва ТРК ""Атриум"""
3,2013-06-01,0,25,2554,1709.05,1.0,DEEP PURPLE Who Do You Think We Are LP,58,Музыка - Винил,"Москва ТРК ""Атриум"""
4,2013-01-15,0,25,2555,1099.00,1.0,DEEP PURPLE 30 Very Best Of 2CD (Фирм.),56,Музыка - CD фирменного производства,"Москва ТРК ""Атриум"""


In [51]:
# Split-out validation dataset
array = sales.values
X = array[:,1:6]
Y = array[:,6]
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.80, random_state=0)

In [52]:
# Spot Check Algorithms
models = []
models.append(('LR', LogisticRegression(solver='liblinear', multi_class='ovr')))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
#models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('SVM', SVC(gamma='auto')))


In [53]:
# evaluate each model in turn
results = []
names = []


In [ ]:
for name, model in models:
    kfold = StratifiedKFold(n_splits=2, random_state=None, shuffle=True)
    cv_results = cross_val_score(model, X_train, Y_train, cv=kfold, scoring='accuracy')
    results.append(cv_results)
    names.append(name)
    print('%s: %f (%f)' % (name, cv_results.mean(), cv_results.std()))


C:\Users\91882\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:667: UserWarning: The least populated class in y has only 1 members, which is less than n_splits=2.
  % (min_groups, self.n_splits)), UserWarning)


In [ ]:
# # Make predictions on validation dataset
# model = SVC(gamma='auto')
# model.fit(X_train, Y_train)
# predictions = model.predict(X_validation)

In [ ]:
# # Evaluate predictions
# print(accuracy_score(Y_validation, predictions))
# print(confusion_matrix(Y_validation, predictions))
# print(classification_report(Y_validation, predictions))